# import libraries

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor

# check library versions
print('numpy version:', np.__version__)
print('pandas version:', pd.__version__)
# print('matplotlib version:', plt.__version__)

# load csv file and check dataframe

In [ ]:
# create dataframe and check if it works
csv_path = "../data/01-01-2012_31-12-2021/Ongevallengegevens/ongevallen.csv" # 2012 to 2021
# csv_path = "../data/01-01-2021_31-12-2021/Ongevallengegevens/ongevallen.csv" # 2021 only
df_accidents = pd.read_csv(csv_path, sep=',')
df_accidents.columns =["VKL_NUMMER","REGNUMMER","PVOPGEM","DATUM_VKL","DAG_CODE","MND_NUMMER","JAAR_VKL","TIJDSTIP","UUR","DDL_ID","AP3_CODE","AP4_CODE","AP5_CODE","ANTL_SLA","ANTL_DOD","ANTL_GZH","ANTL_SEH","ANTL_GOV","ANTL_PTJ","ANTL_TDT","MNE_CODE","AOL_ID","NIVEAUKOP","WSE_ID","WSE_AN","BEBKOM","MAXSNELHD","WVL_ID","WVG_ID","WVG_AN","WDK_ID","WDK_AN","LGD_ID","ZAD_ID","WGD_CODE_1","WGD_CODE_2","BZD_ID_VM1","BZD_ID_VM2","BZD_ID_VM3","BZD_VM_AN","BZD_ID_IF1","BZD_ID_IF2","BZD_ID_IF3","BZD_IF_AN","BZD_ID_TA1","BZD_ID_TA2","BZD_ID_TA3","BZD_TA_AN","JTE_ID","WVK_ID","HECTOMETER","FK_VELD5","HUISNUMMER","GME_ID","GME_NAAM","PVE_CODE","PVE_NAAM","KDD_NAAM","PLT_NAAM","DIENSTCODE","DIENSTNAAM","DISTRCODE","DISTRNAAM","DAGTYPE","IND_ALC","WEEKNR"]
df_accidents.head(10)

In [ ]:
# check how many fields are filled in per column
for i, c in df_accidents.iteritems():
    k = 0
    for j in c:
        if not pd.isna(j):
            k+=1
    print(i, ": ", k)

# drop unnecessary columns

In [ ]:
# test dropping columns
df_test = df_accidents.drop(columns=["PVOPGEM", "DATUM_VKL"])
df_test.head(10)

In [ ]:
# drop columns with less than x non empty in cells
threshold = 100000
df_2 = df_accidents
bad_rows = []
for i, c in df_accidents.iteritems():
    k = 0
    for j in c:
        if not pd.isna(j):
            k+=1
    if k <= threshold:
        bad_rows.append(i)
df_2 = df_2.drop(columns=bad_rows)
df_2.head(10)

In [ ]:
for i, c in df_2.iteritems():
    k = 0
    for j in c:
        if not pd.isna(j):
            k+=1
    print(i, ": ", k)

# plot yearly and apply linear regression

In [ ]:
# plot yearly accidents

min_year = df_2.min(axis='rows', numeric_only=True)[1].astype(int)
max_year = df_2.max(axis='rows', numeric_only=True)[1].astype(int)
year = range(min_year, max_year + 1)
n_accidents = []
for i, j in enumerate(year):
    n = 0
    for k in df_2.iloc[:,2]:
        if j == k:
            n+=1
    n_accidents.insert(i, n)

print(year)
print(n_accidents)
fig, ax = plt.subplots()
ax.plot(year, n_accidents)
plt.show()

In [ ]:
print(year)
print(n_accidents)
year_2 = np.array(year).reshape(-1, 1)
n_accidents_2 = n_accidents
linear_regressor = LinearRegression()
linear_regressor.fit(year_2, n_accidents_2)
Y_pred = linear_regressor.predict(year_2)

In [ ]:
# show proper figure
plt.title("yearly total number of road accidents in the netherlands")
plt.xlabel("year")
plt.ylabel("number of accidents")

plt.plot(year, n_accidents, label="total accidents")
plt.plot(year, Y_pred, color="red", label="trendline")
plt.legend()
plt.show()

# plot monthly and apply linear regression

In [ ]:
print(df_2["PVE_NAAM"].unique())
print(df_2["PVE_CODE"].unique())

In [ ]:
df_4 = df_2
df_4["PVE_NUMR"] = df_4["PVE_CODE"].map( {"ZH" : 1, "NB" : 2, "LB" : 3, "GL" : 4, "FL" : 5, "UT" : 6, "NH" : 7, "FR" : 8, "GR" : 9, "DR" : 10, "ZL" : 11, "OV" : 12} ).astype(int)
df_4.head(3)

In [ ]:
df_5 = df_4[["JAAR_VKL", "PVE_CODE", "PVE_NUMR"]]
df_5.head(5)

In [ ]:
years_6 = df_5["JAAR_VKL"].unique()
provinces_6 = df_5["PVE_CODE"].unique()
provinces_number_6 = df_5["PVE_NUMR"].unique()

df_6 = pd.DataFrame(columns=["YEAR", "PROVINCE", "N_ACCIDENTS"])
i = 0
for j in years_6:
    for k in provinces_6:
        df_6.loc[i] = [j, k, 0]
        i+=1


df_6

In [ ]:
# for i, j in df_5.iterrows():
#     for k, l in df_6.iterrows():
#         if j[0] == l[0] and j[1] == l[1]:
#             df_6.iloc[k, 2] = df_6.iloc[k, 2] +1
#             break

# df_6

In [ ]:
df_6.to_csv("df_6.csv", index=False)

In [ ]:
df_7 = pd.read_csv("df_6.csv", sep=',')
df_7

In [ ]:
target_province = "DR"
sub_df_7 = df_7[df_7["PROVINCE"]==target_province]
sub_df_7

In [ ]:
min_year = df_7["YEAR"].min()
max_year = df_7["YEAR"].max()
year = range(min_year, max_year + 1)
n_accidents = []
for i, j in enumerate(year):
    row = sub_df_7.loc[df_7["YEAR"]==j]
    n_accidents.insert(i, row["N_ACCIDENTS"])
fig, ax = plt.subplots()
ax.plot(year, n_accidents)
plt.show()

In [ ]:
target_province = "GR"
df_7 = df_5.loc[df_5["PVE_CODE"] == target_province]
df_9 = df_7["JAAR_VKL"].value_counts()
df_10 = df_9.reset_index()
df_11 = df_10.sort_values("index", ignore_index=True)
print(df_9)
print(df_10)
print(df_11)

In [ ]:
year = df_11.iloc[:,0]
n_accidents = df_11.iloc[:,1]
year_2 = np.array(year).reshape(-1, 1)
n_accidents_2 = n_accidents
linear_regressor = LinearRegression()
linear_regressor.fit(year_2, n_accidents_2)
Y_pred = linear_regressor.predict(year_2)
plt.title("yearly total number of road accidents in the netherlands")
plt.xlabel("year")
plt.ylabel("number of accidents")

plt.plot(year, n_accidents, label="total accidents in " + target_province)
plt.plot(year, Y_pred, color="red", label="trendline")
plt.legend()
plt.show()

In [ ]:
year = df_11.iloc[:,0]
n_accidents = df_11.iloc[:,1]
year_2 = np.array(year).reshape(-1, 1)
n_accidents_2 = n_accidents
SGD_regression = SGDRegressor()
SGD_regression.fit(year_2, n_accidents_2)
Y_pred = SGD_regression.predict(year_2)
plt.title("yearly total number of road accidents in the netherlands")
plt.xlabel("year")
plt.ylabel("number of accidents")

# plt.plot(year, n_accidents, label="total accidents in " + target_province)
plt.plot(year, Y_pred, color="red", label="trendline")
plt.legend()
plt.show()